In [4]:
from transformers import AutoTokenizer, BertModel, BertConfig
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:

import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
config = BertConfig.from_pretrained("bert-base-uncased")
config.hidden_size




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


768

In [8]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
last_hidden_states.shape



<class 'transformers.tokenization_utils_base.BatchEncoding'>


torch.Size([1, 8, 768])

In [3]:
import pandas as pd
from torch.utils.data import DataLoader
from torch.utils.data import Dataset


class MyData(Dataset):
    
    def __init__(self,dataframe):
        self.df = dataframe
        self.x = df[0]
        self.y = torch.tensor(df[1].values)
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]


df = pd.read_json('./archive/is_train.json')

df[1], uniques = pd.factorize(df[1])

num_classes = len(uniques)
print(num_classes)
print(len(df[1]))

train_dataloader = DataLoader(MyData(df), batch_size=64, shuffle=True)



150
15000


In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

class BertClassification(nn.Module):
    def __init__(self, model_name, num_classes):
        super(BertClassification, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.config = BertConfig.from_pretrained(model_name)
        self.classifier = nn.Linear(in_features=self.config.hidden_size, out_features=num_classes)
        
        torch.nn.init.xavier_normal_(self.classifier.weight.data)
        torch.nn.init.uniform_(self.classifier.bias.data)
        
    def forward(self,inputs):
        output = self.bert(**inputs)
        output = self.classifier(output.last_hidden_state[:,0])
        output = F.sigmoid(output)
        
        return output
        
        

In [15]:
#classifier = BertClassification("bert-base-uncased", num_classes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=classifier.parameters(), lr=1e-6, weight_decay=1e-5)

epoches = 10


classifier.to(device)
classifier.train()

train_loss_avg = []

for epoch in range(epoches):
    train_loss_avg.append(0)
    num_batches = 0
    
    for train_x, train_y in train_dataloader:
        inputs = tokenizer(train_x, padding=True, return_tensors='pt').to(device)
        train_y = train_y.to(device)
        
        output = classifier(inputs)
    
        loss = criterion(output, train_y)
    
        optimizer.zero_grad()
        loss.backward()
    
        optimizer.step()
        
        train_loss_avg[-1] += loss.item()
        num_batches += 1
    
        
    train_loss_avg[-1] /= num_batches
    print('Epoch [%d / %d] average reconstruction error: %f' % (epoch+1, epoches, train_loss_avg[-1]))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda:0
Epoch [1 / 10] average reconstruction error: 5.007173
Epoch [2 / 10] average reconstruction error: 4.930029
Epoch [3 / 10] average reconstruction error: 4.845474
Epoch [4 / 10] average reconstruction error: 4.772070
Epoch [5 / 10] average reconstruction error: 4.712622
Epoch [6 / 10] average reconstruction error: 4.659827
Epoch [7 / 10] average reconstruction error: 4.616968
Epoch [8 / 10] average reconstruction error: 4.578407
Epoch [9 / 10] average reconstruction error: 4.545884
Epoch [10 / 10] average reconstruction error: 4.517409


In [5]:
torch.cuda.empty_cache()

In [28]:
torch.save(classifier, './classifier.pt')
#classifier = torch.load('./classifier.pt')

In [38]:
def predict(model, tokenizer, input):
    return model(tokenizer(input, return_tensors="pt"))

In [27]:
classifier.eval()
classifier.to('cpu')
value = torch.argmax(classifier(tokenizer("turn on cnn", return_tensors='pt'))).item()
uniques[value]



'play_music'